In [193]:
import numpy as np
import pandas as pd
import torch
import sys
import pickle

from sklearn.manifold import TSNE
import plotly.graph_objects as go

from src.models.scale import Scale
import src.visualization.scale_color_filter as scf

# Loading Model and Vocabulary

In [194]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load(f"result_model/cbow_network.pt", map_location=device)

with open('./data/training_data.pkl', 'rb') as file:
        data = pickle.load(file, encoding="latin1")

vocab = data['chords_vocabulary']

c:\Users\Proprio\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:707: UserWarning:

'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)



# Getting Embeddings

In [195]:
# embedding from first model layer
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms

print(f'embeddings shape: {embeddings.shape}')
print(f'embeddings norms shape: {embeddings_norm.shape}')

embeddings shape: (298, 25)
embeddings norms shape: (298, 25)


# Visualization with t-SNE

In [196]:
# get embeddings
embeddings_df = pd.DataFrame(embeddings)

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
chord_suffixes = np.array(list(vocab.suffixes_to_indexes.keys()))
embeddings_df_trans.index = chord_suffixes

In [197]:
colors = ['lightgrey'] * len(chord_suffixes)
scale = Scale('C', 'major')

colors = scf.get_scale_all_triads_filter(colors, chord_suffixes, scale, 'blue')
colors = scf.get_scale_chord_degress(colors, chord_suffixes, 0, 'major', 'green')
colors = scf.get_scale_main_triads_filter(colors, chord_suffixes, scale, 'red')

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=colors)
    )
)

# Similare chords

In [198]:
def get_top_similar(chord_suffix: str, topN: int = 10):
    chord_id = vocab.suffixes_to_indexes[chord_suffix]

    if chord_id == 0:
        print("Out of vocabulary chord")
        return

    chord_vec = embeddings_norm[chord_id]
    chord_vec = np.reshape(chord_vec, (len(chord_vec), 1))
    dists = np.matmul(embeddings_norm, chord_vec).flatten()
    topN_ids = np.argsort(-dists)[1 : topN + 1]

    topN_dict = {}
    for sim_chord_id in topN_ids:
        sim_chord = vocab.indexes_to_chords[sim_chord_id]
        sim_chord_suffix = ''.join(sorted(sim_chord.note_suffixes))
        topN_dict[sim_chord_suffix] = dists[sim_chord_id]

    return topN_dict

In [199]:
for chord, sim in get_top_similar("CEG").items():
    print("{}: {:.3f}".format(chord, sim))

CEF: 0.445
ACD: 0.342
BCD#: 0.318
A#D#G: 0.289
CFG: 0.288
BDG: 0.284
AD#F#: 0.277
ADG: 0.266
CDG: 0.259
ABC: 0.258
